In [ ]:
pip install fake-useragent

In [ ]:
pip install requests-html 


In [ ]:
import requests
import urllib
import pandas as pd
from requests_html import HTML
from requests_html import HTMLSession

In [ ]:
import requests
import urllib
import pandas as pd
from requests_html import HTML
from requests_html import HTMLSession
import numpy as np
from fake_useragent import UserAgent
import re
from urllib.request import Request, urlopen
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import math

from bs4 import BeautifulSoup

def get_text(url):
    try:
        req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urlopen(req,timeout=5).read()
        soup = BeautifulSoup(webpage, "html.parser")
        texts = soup.findAll(text=True)
        res=u" ".join(t.strip() for t in texts if t.parent.name not in ['style', 'script', 'head', 'title', 'meta', '[document]'])
        return(res)
    except:
        return False

def get_source(url):
    """Return the source code for the provided URL. 
    Args: 
        url (string): URL of the page to scrape.
    Returns:
        response (object): HTTP response object from requests_html. 
    """
    try:
        session = HTMLSession()
        response = session.get(url)

       
        return response
    except requests.exceptions.RequestException as e:
        print(e)
def scrape_google(query):

    query = urllib.parse.quote_plus(query)
    response = get_source("https://www.google.co.uk/search?q=" + query)
    links = list(response.html.absolute_links)
    google_domains = ('https://www.google.', 
                      'https://google.', 
                      'https://webcache.googleusercontent.', 
                      'http://webcache.googleusercontent.', 
                      'https://policies.google.',
                      'https://support.google.',
                      'https://maps.google.')

    for url in links[:]:
        if url.startswith(google_domains):
            links.remove(url)

    return links

scrape_google("cars")

In [ ]:
from nltk.data import url2pathname
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
from fake_useragent import UserAgent
import re
from urllib.request import Request, urlopen
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import math

from bs4 import BeautifulSoup


stop_words = set(stopwords.words('english'))
print(len(stop_words))
stop_words


In [ ]:
import nltk
nltk.download('punkt')
# example_sent = get_text('https://en.wikipedia.org/wiki/Machine_learning')
stop_words = set(stopwords.words('english'))

links=scrape_google('cars')

text=[]
for i in links:
  t=get_text(i)
  if t:
    text.append(t)

    word_tokens = word_tokenize(t)
# word_tokens = 

    filtered_sentence = [w for w in word_tokens]

    # print(word_tokens)
    # print('---------------------------------')
    # print(filtered_sentence)

    filtered_sentence = [w for w in word_tokens if not w in stop_words]

    # print(word_tokens)
    # print('---------------------------------')
    # print(filtered_sentence)
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]

    print(word_tokens)
    print('---------------------------------')
    print(filtered_sentence)

In [ ]:
import nltk
from nltk.stem.porter import *
from nltk.stem.snowball import SnowballStemmer
s_stemmer = SnowballStemmer(language='english')
p_stemmer = PorterStemmer()

for word in filtered_sentence:
    if len(word) >= 3:
        print(word+' --> '+p_stemmer.stem(word))
        print(word+' --> '+s_stemmer.stem(word))
u=p_stemmer.stem(word) + s_stemmer.stem(word)


In [ ]:
import pandas as pd
def tf_idf_analysis(keyword):
    v = TfidfVectorizer(min_df=1,analyzer='word',ngram_range=(1,2),stop_words=list(stop_words))
    x = v.fit_transform(text)

    f = pd.DataFrame(x.toarray(), columns = v.get_feature_names_out())
    d=pd.concat([pd.DataFrame(f.mean(axis=0)),pd.DataFrame(f.max(axis=0))],axis=1)
    
    tf=pd.DataFrame((f>0).sum(axis=0))


    d=d.reset_index().merge(tf.reset_index(),on='index',how='left')

    d.columns=['word','average_tfidf','max_tfidf','frequency']

    d['frequency']=round((d['frequency']/len(text))*100)

    return(d)

x= tf_idf_analysis(u)
x = x[x['word'].str.isalpha() & (x['word'].str.len() >= 4)].sort_values('max_tfidf', ascending=False).head(35)
print(x)

In [ ]:
import nltk
import gensim.downloader as api
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
# Download the pre-trained Word2Vec model
model = api.load('word2vec-google-news-300')

word_list = stemmed_words

# Load stop words and create a lemmatizer object
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

# Define a function to preprocess text
def preprocess(text):
    tokens = nltk.word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalpha() and token not in stop_words]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return tokens

# Preprocess the words in the list
word_list_preprocessed = [preprocess(word) for word in word_list]

# Define a function to find the alternative words for a word with high Word2Vec similarity scores
def find_high_similarity_alternative_words(word, threshold=0.6):
    alternatives = []
    if word not in model:
        return alternatives
    for w, score in model.most_similar(word):
        if score > threshold:
            alternatives.append(w)
    return alternatives

# Find the higher ranking alternative words for each word in the list
alternatives_dict = {}
for word in word_list:
    alternatives = find_high_similarity_alternative_words(word)
    alternatives_dict[word] = alternatives

# Print the higher ranking alternative words for each word in the list
for word, alternatives in alternatives_dict.items():
    print(word + ':', alternatives)